In [ ]:
# 데이터 전처리 

import os
import pandas as pd
import geopandas as gpd

def try_encodings(file_path, encodings=None):
    if encodings is None:
        encodings = ["utf-8", "cp949", "euc-kr"]
        
    for enc in encodings:
        try:
            with open(file_path, "r", encoding=enc) as f:
                f.read()
            return enc
        except UnicodeDecodeError:
            continue
        except Exception as e:
            print(f"다른 예외 발생: {e}")
            continue
    raise ValueError(f"파일을 열 수 있는 인코딩을 찾을 수 없습니다: {file_path}")

filenames = {
    #"life_mobility":   "생활이동 데이터(250m).csv",
    "sports_facility": "체육시설 좌표.csv",
    "trash_bin_2024":  "2024_서울시_쓰레기통_좌표.csv",
    "bus_stops":       "서울시 버스정류소 위치정보.csv",
    "parking_lot":     "주차장 좌표.csv",
    "legal_boundary":  "13.서울시_법정경계(시군구).geojson",
    "school":          "서울시_학교_좌표.csv",
    "sidewalk":        "인도4.csv",
    "road":            "24.서울시_도로명주소(도로).geojson"
}

data_path = "./data"   # 데이터가 저장된 폴더 경로
df = {}

for key, filename in filenames.items():
    full_path = os.path.join(data_path, filename)
    if filename.endswith(".csv"):
        enc = try_encodings(full_path)
        # CSV 파일은 pandas로 불러오기
        df[key] = pd.read_csv(full_path,encoding = enc)
    elif filename.endswith(".geojson"):
        # GeoJSON 파일은 geopandas로 불러오기
        df[key] = gpd.read_file(full_path)
    else:
        # 기타 확장자에 대한 처리를 추가적으로 할 수 있음
        print(f"Unsupported file format for {key}: {filename}")


,ETL_YMD,O_CELL_ID,O_CELL_X,O_CELL_Y,O_CELL_TP,D_CELL_ID,D_CELL_X,D_CELL_Y,D_CELL_TP,ST_TIME_CD,...,FEML_40_CNT,FEML_45_CNT,FEML_50_CNT,FEML_55_CNT,FEML_60_CNT,FEML_65_CNT,FEML_70_CNT,FEML_75_CNT,FEML_80_CNT,FEML_85_CNT
0,20240318,다사31004675,931125,1946875,2,다사31004500,931125,1945125,0,20,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,20240315,다사44755000,944875,1950125,2,다사58254750,958375,1947625,1,6,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,20240314,다사54004200,954125,1942125,0,다사64254900,964375,1949125,2,21,...,0.0,0.0,3.34,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,20240319,다사52255125,952375,1951375,1,다사66505925,966625,1959375,0,19,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20240311,다사54251650,954375,1916625,1,다사52752175,952875,1921875,0,11,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [1]:
# 각 데이터프레임의 컬럼 이름을 출력하는 함수
def print_column_names(dataframes):
    """
    딕셔너리에 저장된 데이터프레임들의 컬럼 이름을 출력합니다.
    """
    for key, dataframe in dataframes.items():
        print(f"Dataset '{key}'의 컬럼 이름:")
        try:
            print(dataframe.columns.tolist())
        except AttributeError:
            print("해당 데이터는 데이터프레임 형식이 아닙니다.")
        print("-" * 40)

In [17]:
from shapely.geometry import Point

def create_geodataframe(df, lon_col="longitude", lat_col="latitude", crs="EPSG:4326"):
    """
    Pandas DataFrame에서 GeoPandas GeoDataFrame을 생성하는 함수.
    - 해당 컬럼이 없으면 원본 DataFrame을 그대로 반환합니다.
    
    Parameters:
    - df: 원본 DataFrame
    - lon_col: 경도 컬럼 이름
    - lat_col: 위도 컬럼 이름
    - crs: 좌표계 (기본값: WGS84, "EPSG:4326")
    
    Returns:
    - GeoDataFrame 또는 원본 DataFrame
    """
    if lon_col in df.columns and lat_col in df.columns:
        # Point 객체로 geometry 생성
        geometry = [Point(xy) for xy in zip(df[lon_col], df[lat_col])]
        # GeoDataFrame 생성
        gdf = gpd.GeoDataFrame(df, geometry=geometry)
        gdf.set_crs(crs, inplace=True)  # 좌표계 설정
        return gdf
    else:
        # 경도, 위도 컬럼이 없으면 원본 DataFrame 반환
        print(f"경고: '{lon_col}' 또는 '{lat_col}' 컬럼이 존재하지 않습니다. 변환 생략.")
        return df
    


# df 딕셔너리 내 모든 데이터프레임 처리
#for key, dataframe in df.items():
#    print(f"Processing dataset: {key}")
#    df[key] = create_geodataframe(dataframe)



Processing dataset: life_mobility
경고: 'longitude' 또는 'latitude' 컬럼이 존재하지 않습니다. 변환 생략.
Processing dataset: sports_facility
Processing dataset: trash_bin_2024
Processing dataset: bus_stops
경고: 'longitude' 또는 'latitude' 컬럼이 존재하지 않습니다. 변환 생략.
Processing dataset: parking_lot
Processing dataset: legal_boundary
경고: 'longitude' 또는 'latitude' 컬럼이 존재하지 않습니다. 변환 생략.
Processing dataset: school
Processing dataset: sidewalk
경고: 'longitude' 또는 'latitude' 컬럼이 존재하지 않습니다. 변환 생략.
Processing dataset: road
경고: 'longitude' 또는 'latitude' 컬럼이 존재하지 않습니다. 변환 생략.


In [21]:
def convert_to_geodataframe(df, lon_col, lat_col, crs="EPSG:4326"):
    """
    위도와 경도 데이터를 기반으로 Pandas DataFrame을 GeoDataFrame으로 변환합니다.
    
    Parameters:
    - df: Pandas DataFrame
    - lon_col: 경도 컬럼 이름
    - lat_col: 위도 컬럼 이름
    - crs: 좌표계 (기본값: EPSG:4326)
    
    Returns:
    - GeoDataFrame
    """
    # Point 객체로 geometry 생성
    geometry = [Point(xy) for xy in zip(df[lon_col], df[lat_col])]
    # GeoDataFrame 생성
    gdf = gpd.GeoDataFrame(df, geometry=geometry)
    gdf.set_crs(crs, inplace=True)  # 좌표계 설정
    return gdf
df['bus_stops'] = convert_to_geodataframe(df['bus_stops'],lon_col = 'X좌표',lat_col = 'Y좌표')
df['bus_stops'].head()

,노드 ID,정류소번호,정류소명,X좌표,Y좌표,정류소 타입,geometry
0,100000001,1001,종로2가사거리,126.987752,37.569808,중앙차로,POINT (126.98775 37.56981)
1,100000002,1002,창경궁.서울대학교병원,126.996522,37.579433,중앙차로,POINT (126.99652 37.57943)
2,100000003,1003,명륜3가.성대입구,126.998251,37.582581,중앙차로,POINT (126.99825 37.58258)
3,100000004,1004,종로2가.삼일교,126.987613,37.568579,중앙차로,POINT (126.98761 37.56858)
4,100000005,1005,혜화동로터리.여운형활동터,127.001744,37.586243,중앙차로,POINT (127.00174 37.58624)


In [22]:
re_cols = {'X좌표':'longitude','Y좌표':'latitude'}
df['bus_stops'].rename(columns = re_cols,inplace = True)
df['bus_stops'].head()

,노드 ID,정류소번호,정류소명,longitude,latitude,정류소 타입,geometry
0,100000001,1001,종로2가사거리,126.987752,37.569808,중앙차로,POINT (126.98775 37.56981)
1,100000002,1002,창경궁.서울대학교병원,126.996522,37.579433,중앙차로,POINT (126.99652 37.57943)
2,100000003,1003,명륜3가.성대입구,126.998251,37.582581,중앙차로,POINT (126.99825 37.58258)
3,100000004,1004,종로2가.삼일교,126.987613,37.568579,중앙차로,POINT (126.98761 37.56858)
4,100000005,1005,혜화동로터리.여운형활동터,127.001744,37.586243,중앙차로,POINT (127.00174 37.58624)


In [24]:
print_column_names(df)
df['sports_facility'].head()

Dataset 'life_mobility'의 컬럼 이름:
['ETL_YMD', 'O_CELL_ID', 'O_CELL_X', 'O_CELL_Y', 'O_CELL_TP', 'D_CELL_ID', 'D_CELL_X', 'D_CELL_Y', 'D_CELL_TP', 'ST_TIME_CD', 'FNS_TIME_CD', 'IN_FORN_DIV_NM', 'MOVE_PURPOSE', 'MOVE_DIST', 'MOVE_TIME', 'MALE_00_CNT', 'MALE_10_CNT', 'MALE_20_CNT', 'MALE_30_CNT', 'MALE_35_CNT', 'MALE_40_CNT', 'MALE_45_CNT', 'MALE_50_CNT', 'MALE_55_CNT', 'MALE_60_CNT', 'MALE_65_CNT', 'MALE_70_CNT', 'MALE_75_CNT', 'MALE_80_CNT', 'MALE_85_CNT', 'FEML_00_CNT', 'FEML_10_CNT', 'FEML_15_CNT', 'FEML_20_CNT', 'FEML_25_CNT', 'FEML_30_CNT', 'FEML_35_CNT', 'FEML_40_CNT', 'FEML_45_CNT', 'FEML_50_CNT', 'FEML_55_CNT', 'FEML_60_CNT', 'FEML_65_CNT', 'FEML_70_CNT', 'FEML_75_CNT', 'FEML_80_CNT', 'FEML_85_CNT']
----------------------------------------
Dataset 'sports_facility'의 컬럼 이름:
['Unnamed: 0', '시설주소', 'latitude', 'longitude', 'geometry']
----------------------------------------
Dataset 'trash_bin_2024'의 컬럼 이름:
['Unnamed: 0', 'address', 'latitude', 'longitude', 'geometry']
---------------

,Unnamed: 0,시설주소,latitude,longitude,geometry
0,0,서울특별시 구로구 오류로 36-25,37.488639,126.841743,POINT (126.84174 37.48864)
1,1,서울특별시 강서구 공항대로36길 74,37.554982,126.836344,POINT (126.83634 37.55498)
2,2,서울특별시 강서구 양천로61길 101,37.559960,126.865560,POINT (126.86556 37.55996)
3,3,서울특별시 강서구 양천로61길 101,37.559960,126.865560,POINT (126.86556 37.55996)
4,4,서울특별시 강서구 양천로61길 101,37.559960,126.865560,POINT (126.86556 37.55996)


In [25]:
drop_cols = 'Unnamed: 0'
for key, dataframe in df.items():
    if drop_cols in dataframe.columns:
        dataframe.drop(columns=drop_cols, inplace=True)

print_column_names(df)


Dataset 'life_mobility'의 컬럼 이름:
['ETL_YMD', 'O_CELL_ID', 'O_CELL_X', 'O_CELL_Y', 'O_CELL_TP', 'D_CELL_ID', 'D_CELL_X', 'D_CELL_Y', 'D_CELL_TP', 'ST_TIME_CD', 'FNS_TIME_CD', 'IN_FORN_DIV_NM', 'MOVE_PURPOSE', 'MOVE_DIST', 'MOVE_TIME', 'MALE_00_CNT', 'MALE_10_CNT', 'MALE_20_CNT', 'MALE_30_CNT', 'MALE_35_CNT', 'MALE_40_CNT', 'MALE_45_CNT', 'MALE_50_CNT', 'MALE_55_CNT', 'MALE_60_CNT', 'MALE_65_CNT', 'MALE_70_CNT', 'MALE_75_CNT', 'MALE_80_CNT', 'MALE_85_CNT', 'FEML_00_CNT', 'FEML_10_CNT', 'FEML_15_CNT', 'FEML_20_CNT', 'FEML_25_CNT', 'FEML_30_CNT', 'FEML_35_CNT', 'FEML_40_CNT', 'FEML_45_CNT', 'FEML_50_CNT', 'FEML_55_CNT', 'FEML_60_CNT', 'FEML_65_CNT', 'FEML_70_CNT', 'FEML_75_CNT', 'FEML_80_CNT', 'FEML_85_CNT']
----------------------------------------
Dataset 'sports_facility'의 컬럼 이름:
['시설주소', 'latitude', 'longitude', 'geometry']
----------------------------------------
Dataset 'trash_bin_2024'의 컬럼 이름:
['address', 'latitude', 'longitude', 'geometry']
----------------------------------------
Da